In [5]:
import pyexiv2
import requests
import json

In [21]:
# Classifica a imagem por tags enviando pelo tagbox e recebendo as tags
def classify_file(filename, file):
    response = requests.post('http://localhost:8080/tagbox/check', files={'file': (filename, file)})
    body = json.loads(response.content)
    return body['success'], body['tags']

# Modifica o exif das imagens para incluir as tags
def modify_tags(filename, tags):
    with open(filename, 'rb+') as file:
        with pyexiv2.ImageData(file.read()) as img:
            metadata = img.read_xmp()
            img.modify_xmp({'Xmp.dc.subject': tags})
            file.seek(0)
            file.write(img.get_bytes())

def get_tags(file):
    with pyexiv2.ImageData(file.read()) as img:
        return img.read_xmp()['Xmp.dc.subject']

# Le arquivo e manda para classificação
def classify_by_filename(filename):
    with open(filename, 'rb+') as file:
        res, tags = classify_file(filename, file)
        cleaned_tags = [tag['tag'] for tag in tags]
        if not res:
            raise ValueError('Impossible to classify image!')
        return cleaned_tags, file

In [22]:
# Vamos testar a classificação com 3 imagens
anime_res, anime_file = classify_by_filename("resources/anime.png")
woman_res, woman_file = classify_by_filename("resources/woman.jpg")
montains_res, montains_file = classify_by_filename("resources/place.jpg")

print("Anime:", ", ".join(anime_res))
print("Woman:",  ", ".join(woman_res))
print("Montains:", ", ".join(montains_res))

Anime: Cartoon, Anime, Clothing, Illustration, Adult
Woman: Sitting, Leg, Limb, Clothing, Photo shoot, Furniture, Footwear
Montains: Mountain, Hill, Landscape, Plateau, Ridge, Horizon, Soil, Rock, Geology, Morning


In [32]:
# Agora já podemos escrever o exif da classificação nas 3 imagens
modify_tags("resources/anime.png", anime_res)
modify_tags("resources/woman.jpg", woman_res)
modify_tags("resources/place.jpg", montains_res)

In [33]:
with open("resources/anime.png", 'rb+') as file:
    with pyexiv2.ImageData(file.read()) as img:
        print(img.read_xmp()['Xmp.dc.subject'])

['Cartoon', 'Anime', 'Clothing', 'Illustration', 'Adult']
